In [82]:
import f90nml as nml
import numpy as np
import xarray as xr
import salishsea_cmd.api
import yaml

In [ ]:
# Used to estimate a distance between two model outputs (with the same coordinate space)
def SmeltResultDif(result_1_dir, result_2_dir, tracer_filename = 'SS5x5_1h_20041019_20041023_ptrc_T.nc'):
    ds1 = xr.open_dataset(result_1_dir + '/' +tracer_filename)
    ds2 = xr.open_dataset(result_2_dir +'/' + tracer_filename)
    
    #Picked MICZ pretty much arbitrarily
    dif = ((np.square(ds1.MICZ.values - ds2.MICZ.values)).sum())**0.5 #RMS difference (should probably normalise by number of data points)
    return(dif)

In [58]:
reference_namelist = '/data/jpetrie/MEOPAR/SalishSea/results/d9f69a38-160e-11e6-9b98-0025909a8461/namelist_pisces_cfg'
new_namelist_directory = '/data/jpetrie/MEOPAR/SS-run-sets/SS-SMELT/namelists/modified_namelists/'
reference_result = '/data/jpetrie/MEOPAR/SalishSea/results/d9f69a38-160e-11e6-9b98-0025909a8461'

In [ ]:
reference_nml = f90nml.read(reference_namelist) # creates a dictionary-like object with contents of namelist

In [73]:
grad = dict.fromkeys(reference_nml, 0) # create dictionary filled with zeros 
step_size = 1e-6

# Calculate gradient
for paramtype in reference_nml:
    for param in reference_nml[paramtype]:
        print(paramtype + "  " + param)
        
        if(not isinstance(reference_nml[paramtype][param],list)):
            #some values are lists- need something different for them
            patch_nml = {paramtype: {param: reference_nml[paramtype][param] + step_size}} # change parameter slightly 
            modified_nml_file = new_namelist_directory + 'mod_' + paramtype + '_' + param

            # creates a new namelist file identical to original, but with the value specified by patch_nml changed
            f90nml.patch(reference_namelist,patch_nml, modified_nml_file) 
            #### Call Salishsea using new namelist

            #dif = SmeltResultDif(reference_result, new_result)
            #grad[paramtype][param] = dif/step_size


nrdttrc
zz_rate_r_diat
zz_rate_r_myri
zz_rate_r_nano
zz_rate_maxtemp_diat
zz_rate_maxtemp_myri
zz_rate_maxtemp_nano
zz_rate_temprange_diat
zz_rate_temprange_myri
zz_rate_temprange_nano
zz_rate_iopt_diat
zz_rate_iopt_myri
zz_rate_iopt_nano
zz_rate_gamma_diat
zz_rate_gamma_myri
zz_rate_gamma_nano
zz_rate_k_si_diat
zz_rate_k_si_myri
zz_rate_k_si_nano
zz_rate_gamma_o_diat
zz_rate_gamma_o_myri
zz_rate_gamma_o_nano
zz_rate_kapa_diat
zz_rate_kapa_myri
zz_rate_kapa_nano
zz_rate_k_diat
zz_rate_k_myri
zz_rate_k_nano
zz_rate_n_x_diat
zz_rate_n_x_myri
zz_rate_n_x_nano
zz_rate_n_o_diat
zz_rate_n_o_myri
zz_rate_n_o_nano
zz_rate_si_ratio_diat
zz_rate_si_ratio_myri
zz_rate_si_ratio_nano
zz_rate_pico_rm
zz_frac_waste_fnm_nh
zz_frac_waste_fnm_don
zz_frac_waste_fnm_pon
zz_frac_waste_fnm_ref
zz_frac_waste_fnm_bsi
zz_rate_pico_si_ratio
zz_rate_myri_rm
zz_frac_waste_nnm_nh
zz_frac_waste_nnm_don
zz_frac_waste_nnm_pon
zz_frac_waste_nnm_ref
zz_frac_waste_nnm_bsi
zz_rate_myri_si_ratio
zz_rate_micro_rm
zz_frac_w

In [71]:
isinstance(nml['nampismezo']['zz_rate_mesozoo_sumpeakwid'], list) 

True

In [92]:
stream = open('/data/jpetrie/MEOPAR/SS-run-sets/SS-SMELT/jpetrie/SMELT5x5test.yaml', 'r')    # 'document.yaml' contains a single YAML document.

run_desc = yaml.load(stream)

In [96]:
run_desc

{'MPI decomposition': '1x1',
 'config_name': 'SOG5x5test',
 'email': 'jpetrie@eos.ubc.ca',
 'forcing': {'initial_green': {'link to': 'initial_green/'},
  'initial_strat': {'link to': 'initial_strat/'}},
 'grid': {'bathymetry': 'bathy_downonegrid.nc',
  'coordinates': 'coordinates_seagrid_SalishSea.nc'},
 'namelists': {'namelist_cfg': ['/data/jpetrie/MEOPAR/SS-run-sets/SS-SMELT/namelists/namelist_cfg_5x5_NewIC'],
  'namelist_pisces_cfg': ['/data/jpetrie/MEOPAR/SS-run-sets/SS-SMELT/namelists/namelist_pisces_cfg_5x5_NewIC'],
  'namelist_top_cfg': ['/data/jpetrie/MEOPAR/SS-run-sets/SS-SMELT/namelists/namelist_top_cfg_5x5_NewIC']},
 'output': {'XIOS servers': 1,
  'domain': '/data/jpetrie/MEOPAR/SS-run-sets/SS-SMELT/iofiles/domain_def.xml',
  'fields': '/data/jpetrie/MEOPAR/SS-run-sets/SS-SMELT/iofiles/field_def.xml',
  'separate XIOS server': True},
 'paths': {'NEMO-code': '/data/jpetrie/MEOPAR/NEMO-3.6-code/',
  'XIOS': '/data/jpetrie/MEOPAR/XIOS/',
  'forcing': '/data/jpetrie/MEOPAR/NEMO

In [95]:
salishsea_cmd.api.run_in_subprocess("test_salishsea_api", run_desc, '/data/jpetrie/MEOPAR/SS-run-sets/SS-SMELT/iofiles/iodef_bio_1hr.xml', '/data/jpetrie/MEOPAR/SalishSea/results/test_salishsea_api')

salishsea_cmd.api ERROR: subprocess ['salishsea', 'run', 'test_salishsea_api_subprocess_run.yaml', '/data/jpetrie/MEOPAR/SS-run-sets/SS-SMELT/iofiles/iodef_bio_1hr.xml', '/data/jpetrie/MEOPAR/SalishSea/results/test_salishsea_api'] failed with return code 1
salishsea_cmd.api ERROR: salishsea_cmd.run INFO: Created run directory /data/jpetrie/MEOPAR/SalishSea/results/bf311930-170d-11e6-8bbe-10c37b4d98e4
salishsea_cmd.api ERROR: salishsea ERROR: [Errno 2] No such file or directory: 'qsub'
